In [5]:
# from selenium import webdriver
# from selenium.webdriver.support.ui import Select
import re
import pandas as pd
import time

import os
from os import listdir

In [7]:
##### read directory
path = "/home/totoy/Downloads/Legis/legis/dump/17/" #assumes na nasa loob ng bills folder lahat ng files
list_files = os.listdir(path)

##### create an empty dataframe

fields = ['bill_number',#
 'ra_number',#
 'adopted_number', #
 'concurrent_resolution', #
 'full_title',#
 'abstract', #
 'short_title', #
 'date_filed',#
 'significance', #
 'nature', #
 'committee', #
 'date_urgent', #
 'principal',#
 'co_authors_journal',
 'co_authors_committee',
 'co_authors_final',
 'authors_withdraw',
 'committee_actions',
 'committee_rules',
 'second_reading',
 'third_reading',
 'senate_actions',
 'conference_info',
 'president_actions']

df = pd.DataFrame(columns = fields) 

counter = 0
##### parse each file
for textfile in list_files:
    counter += 1
    with open(path+textfile) as file:
        table_cells = [line.rstrip() for line in file]
    
    i=-1
    bill_number = ""
    ra_number = ""
    adopted_number = ""
    concurrent_resolution = ""
    full_title = ""
    abstract = ""
    short_title = ""
    date_filed = ""
    significance = ""
    nature = ""
    committee = ""
    date_urgent = ""

    principal = []
    co_author = ""
    co_authors_journal = []
    co_authors_committee = []
    co_authors_final = []
    authors_withdraw = []
    
    committee_actions = []
    committee_rules = []
    second_reading = []
    third_reading = []
    senate_actions = []
    conference_info = []
    president_actions = []

    for cells in table_cells:
        i += 1
        if re.search(re.compile(r'House Bill/Resolution NO. '), cells) != None:
            bill_number = re.findall('House Bill/Resolution NO. (.*\'?)', cells)[0].strip()
            continue
        if cells[:17] == 'REPUBLIC ACT NO. ':
            ra_number = re.findall('REPUBLIC ACT NO. ', cells)[0].strip()
            continue
        if re.search(re.compile(r'ADOPTED AS RESOLUTION NO. '), cells) != None:
            adopted_number = re.findall('ADOPTED AS RESOLUTION NO. (.*\'?)', cells)[0].strip()
            continue
        if re.search(re.compile(r'COUNTERPART HOUSE BILL/CONCURRENT RESOLUTION: '), cells) != None:
            concurrent_resolution = re.findall('COUNTERPART HOUSE BILL/CONCURRENT RESOLUTION: (.*\'?)', cells)[0].strip()
            continue
        if re.search(re.compile(r'FULL TITLE : '), cells) != None:
            full_title = re.findall('FULL TITLE : (.*\'?)', cells)[0].strip()
            continue
        if re.search(re.compile(r'ABSTRACT : '), cells) != None:
            abstract = re.findall('ABSTRACT : (.*\'?)', cells)[0].strip()
            continue
        if re.search(re.compile(r'SHORT TITLE : '), cells) != None:
            short_title = re.findall('SHORT TITLE : (.*\'?)', cells)[0].strip()
            continue
        if re.search(re.compile(r'ALIAS NAME : '), cells) != None:
            short_title = re.findall('ALIAS NAME : (.*\'?)', cells)[0].strip()
            continue
        if re.search(re.compile(r'PRINCIPAL AUTHOR/S : '), cells) != None:
            principal = re.findall('PRINCIPAL AUTHOR/S : (.*\'?)', cells)[0]
            principal = re.sub(", M.D.,", ' M.D.,', principal)
            principal = principal.split(",")
            principal = [(x+","+y).strip() for x,y in zip(principal[0::2], principal[1::2])]
            continue
        if re.search(re.compile(r'DATE FILED : '), cells) != None:
            date_filed = re.findall('DATE FILED : (.*\'?)', cells)[0].strip()
            continue
        if re.search(re.compile(r'SIGNIFICANCE: '), cells) != None:
            significance = re.findall('SIGNIFICANCE: (.*\'?)', cells)[0].strip()
            continue
        if re.search(re.compile(r'NATURE : '), cells) != None:
            nature = re.findall('NATURE : (.*\'?)', cells)[0].strip()
            continue
        if cells == "CO-AUTHORS (Journal Entries) :":
            for remaining in table_cells[i+1:]:
                if re.search(re.compile(r'^[0-9]+\. '), remaining) != None:
                    co_author = re.findall('[0-9]+\.(.*\'?)\(.*\)$', remaining)[0].strip()
                    co_authors_journal.append(co_author)
                else:
                    break
            continue
        if cells == "AUTHORS(Committee Report) :":
            for remaining in table_cells[i+1:]:
                if re.search(re.compile(r'^[0-9]+\. '), remaining) != None:
                    co_author = re.findall('[0-9]+\.(.*\'?)', remaining)[0].strip()
                    co_authors_committee.append(co_author)
                else:
                    break
        if cells == "AUTHORS(Final/Third Reading) :":
            for remaining in table_cells[i+1:]:
                if re.search(re.compile(r'^[0-9]+\. '), remaining) != None:
                    co_author = re.findall('[0-9]+\.(.*\'?)', remaining)[0].strip()
                    co_authors_final.append(co_author)
                else:
                    break
        if cells == "AUTHORSHIP WITHDRAWALS:":
            for remaining in table_cells[i+1:]:
                if remaining == "()" or remaining == 'ACTIONS TAKEN BY THE COMMITTEE' or remaining == 'ACTIONS TAKEN BY THE COMMITTEE ON RULES' or remaining[:45] == 'COUNTERPART HOUSE BILL/CONCURRENT RESOLUTION:' or remaining[:25] == 'DATE CERTIFIED AS URGENT:': break
                co_author = re.findall('(.*\'?)\(.*\)$', remaining)[0].strip()
                authors_withdraw.append(co_author)
        if re.search(re.compile(r'DATE CERTIFIED AS URGENT:'), cells) != None:
            date_urgent = re.findall('DATE CERTIFIED AS URGENT:(.*\'?)', cells)[0].strip()
            continue
        if cells == "ACTIONS TAKEN BY THE COMMITTEE":
            for remaining in table_cells[i+1:]:
                if remaining == "ACTIONS TAKEN BY THE COMMITTEE ON RULES": break
                committee_actions.append(remaining)
        if cells == "ACTIONS TAKEN BY THE COMMITTEE ON RULES":
            for remaining in table_cells[i+1:]:
                if remaining[:29] == "REFERRAL TO THE COMMITTEE ON ":
                    committee = remaining[29:][:-14]
                if remaining == "SECOND READING INFORMATION": break
                committee_rules.append(remaining)
        if cells == "SECOND READING INFORMATION":
            for remaining in table_cells[i+1:]:
                if remaining == "THIRD READING INFORMATION": break
                second_reading.append(remaining)
        if cells == "THIRD READING INFORMATION":
            for remaining in table_cells[i+1:]:
                if remaining == "ACTIONS TAKEN BY THE SENATE/HOUSE": break
                third_reading.append(remaining)
        if cells == "ACTIONS TAKEN BY THE SENATE/HOUSE":
            for remaining in table_cells[i+1:]:
                if remaining == "CONFERENCE COMMITTEE INFORMATION": break
                senate_actions.append(remaining)
        if cells == "CONFERENCE COMMITTEE INFORMATION":
            for remaining in table_cells[i+1:]:
                if remaining == "ACTIONS TAKEN BY THE PRESIDENT": break
                conference_info.append(remaining)
        if cells == "ACTIONS TAKEN BY THE PRESIDENT":
            for remaining in table_cells[i+1:]:
                president_actions.append(remaining)
        
    data = [[bill_number,
            ra_number,
            adopted_number,
            concurrent_resolution,
            full_title,
            abstract,
            short_title,
            date_filed,
            significance,
            nature,
            committee,
            date_urgent,
            principal,
            co_authors_journal,
            co_authors_committee,
            co_authors_final,
            authors_withdraw,
            committee_actions,
            committee_rules,
            second_reading,
            third_reading,
            senate_actions,
            conference_info,
            president_actions]]
    
    data = pd.DataFrame(data, columns = fields)
    
    df = pd.concat([df, data])
    
    print(counter, "out of", len(list_files))

1 out of 8
2 out of 8
3 out of 8
4 out of 8
5 out of 8
6 out of 8
7 out of 8
8 out of 8


In [10]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [18]:
df.reset_index(inplace=True)

In [19]:
df

,index,bill_number,ra_number,adopted_number,concurrent_resolution,full_title,abstract,short_title,date_filed,significance,nature,committee,date_urgent,principal,co_authors_journal,co_authors_committee,co_authors_final,authors_withdraw,committee_actions,committee_rules,second_reading,third_reading,senate_actions,conference_info,president_actions
0,0,HB00003,,,,"AN ACT GRANTING PRESIDENT RODRIGO ROA DUTERTE,...",Seeks to address the immense traffic congestio...,Traffic Crisis Act Of 2016,2016-06-30,NATIONAL,Reform,TRANSPORTATION,,"[FARIÑAS, RODOLFO C., ALVAREZ, PANTALEON D., A...","[Nieto, Villar, Gonzales (A.P.), Del Mar, Samb...",[],[],[],"[COMMITEE HEARINGS/ACTIONS:, 1. Under de...",[REFERRAL TO THE COMMITTEE ON TRANSPORTATION O...,[],[],[],[],[]
1,0,HB00008,,,,AN ACT INSTITUTIONALIZING A CRIMINAL INVESTIGA...,Institutionalizes a criminal investigation sys...,,2016-06-30,NATIONAL,Reform,JUSTICE,,"[FARIÑAS, RODOLFO C., ALVAREZ, PANTALEON D., A...","[Roque (H.), Nieto, Bataoil, Tan (A.), Villanu...",[],[],[],[MOTHER BILL: HB06574],[REFERRAL TO THE COMMITTEE ON JUSTICE ON 2016-...,[],[],[],[],[]
2,0,HB00005,,,,AN ACT PROVIDING FOR STIFFER PENALTIES FOR PER...,Provides stiffer penalty to persons driving un...,,2016-06-30,NATIONAL,Reform,TRANSPORTATION,,"[FARIÑAS, RODOLFO C., ALVAREZ, PANTALEON D., A...","[Nieto, Gonzales (A.P.), Alvarez (F.), Bataoil...",[],[],[],"[COMMITEE HEARINGS/ACTIONS:, 1. on 2016-...",[REFERRAL TO THE COMMITTEE ON TRANSPORTATION O...,[],[],[],[],[]
3,0,HB00007,,,,AN ACT NAMING THE LAOAG BY-PASS ROAD AS THE RO...,,,2016-06-30,LOCAL,Others,PUBLIC WORKS AND HIGHWAYS,,"[QUIMBO, ROMERO ""MIRO"" S.]",[Cosalan],[],[],[],"[COMMITEE HEARINGS/ACTIONS:, 1. Measure ...",[REFERRAL TO THE COMMITTEE ON PUBLIC WORKS AND...,[],[],[],[],[]
4,0,HB00002,,,,"AN ACT AMENDING REPUBLIC ACT NO. 9344, AS AMEN...",Reverts back the minimum age of criminal liabi...,Minimum Age Of Criminal Responsibility Act,2016-06-30,NATIONAL,Reform,JUSTICE,,"[CASTRO, FREDENIL ""FRED"" H., ALVAREZ, PANTALEO...","[Bataoil, Garcia (G.), Mercado, Cosalan, Anton...",[],[],[Gullas],[MOTHER BILL: HB08858],[REFERRAL TO THE COMMITTEE ON JUSTICE ON 2016-...,[],[],[],[],[]
5,0,HB00004,,,,AN ACT CREATING CIRCUIT CRIMINAL COURTS GRANTI...,Creates Circuit Criminal Courts in all judicia...,Circuit Criminal Courts Act Of 2016,2016-06-30,NATIONAL,Reform,JUSTICE,,"[FARIÑAS, RODOLFO C., ALVAREZ, PANTALEON D., A...","[Nieto, Bataoil, Villanueva, Sy-Alvarado, Espi...",[],[],[],[],[REFERRAL TO THE COMMITTEE ON JUSTICE ON 2016-...,[],[],[],[],[]
6,0,HB00001,,,,AN ACT IMPOSING THE DEATH PENALTY ON CERTAIN H...,Imposes death penalty for heinous crimes with ...,Death Penalty Law,2016-06-30,NATIONAL,Reform,JUSTICE,,"[CASTRO, FREDENIL ""FRED"" H., ALVAREZ, PANTALEO...","[Bataoil, Garcia (G.), Mercado, Cosalan, Yap (...",[],[],[Casilao],[MOTHER BILL: HB04727],[REFERRAL TO THE COMMITTEE ON JUSTICE ON 2016-...,[],[],[],[],[]
7,0,HB00006,,,,AN ACT ENHANCING PUBLIC SAFETY AT THE LOCAL LE...,Seeks to enhance public order and safety at th...,Public Safety Act,2016-06-30,NATIONAL,Reform,LOCAL GOVERNMENT,,"[FARIÑAS, RODOLFO C., ALVAREZ, PANTALEON D., A...","[Roque (H.), Nieto, Bataoil, Cosalan, Tan (A.)...",[],[],[],[MOTHER BILL: HB07878],[REFERRAL TO THE COMMITTEE ON LOCAL GOVERNMENT...,[],[],[],[],[]


In [1]:
df.loc[0,'president_actions']

NameError: name 'df' is not defined

['Roque (H.)',
 'Nieto',
 'Bataoil',
 'Cosalan',
 'Tan (A.)',
 'Villanueva',
 'Floirendo',
 'Sy-Alvarado',
 'Tambunting',
 'Fortun',
 'Espino',
 'Mercado',
 'Dalipe',
 'Violago',
 'Catamco',
 'Revilla',
 'Bordado',
 'Alvarez (F.)',
 'Sarmiento (E.M.)',
 'Sangcopan']

In [25]:
df

,index,bill_number,ra_number,adopted_number,concurrent_resolution,full_title,abstract,short_title,date_filed,significance,nature,committee,date_urgent,principal,co_authors_journal,co_authors_committee,co_authors_final,authors_withdraw,committee_actions,committee_rules,second_reading,third_reading,senate_actions,conference_info,president_actions
0,0,HB00003,,,,"AN ACT GRANTING PRESIDENT RODRIGO ROA DUTERTE,...",Seeks to address the immense traffic congestio...,Traffic Crisis Act Of 2016,2016-06-30,NATIONAL,Reform,TRANSPORTATION,,"[FARIÑAS, RODOLFO C., ALVAREZ, PANTALEON D., A...","[Nieto, Villar, Gonzales (A.P.), Del Mar, Samb...",[],[],[],"[COMMITEE HEARINGS/ACTIONS:, 1. Under de...",[REFERRAL TO THE COMMITTEE ON TRANSPORTATION O...,[],[],[],[],[]
1,0,HB00008,,,,AN ACT INSTITUTIONALIZING A CRIMINAL INVESTIGA...,Institutionalizes a criminal investigation sys...,,2016-06-30,NATIONAL,Reform,JUSTICE,,"[FARIÑAS, RODOLFO C., ALVAREZ, PANTALEON D., A...","[Roque (H.), Nieto, Bataoil, Tan (A.), Villanu...",[],[],[],[MOTHER BILL: HB06574],[REFERRAL TO THE COMMITTEE ON JUSTICE ON 2016-...,[],[],[],[],[]
2,0,HB00005,,,,AN ACT PROVIDING FOR STIFFER PENALTIES FOR PER...,Provides stiffer penalty to persons driving un...,,2016-06-30,NATIONAL,Reform,TRANSPORTATION,,"[FARIÑAS, RODOLFO C., ALVAREZ, PANTALEON D., A...","[Nieto, Gonzales (A.P.), Alvarez (F.), Bataoil...",[],[],[],"[COMMITEE HEARINGS/ACTIONS:, 1. on 2016-...",[REFERRAL TO THE COMMITTEE ON TRANSPORTATION O...,[],[],[],[],[]
3,0,HB00007,,,,AN ACT NAMING THE LAOAG BY-PASS ROAD AS THE RO...,,,2016-06-30,LOCAL,Others,PUBLIC WORKS AND HIGHWAYS,,"[QUIMBO, ROMERO ""MIRO"" S.]",[Cosalan],[],[],[],"[COMMITEE HEARINGS/ACTIONS:, 1. Measure ...",[REFERRAL TO THE COMMITTEE ON PUBLIC WORKS AND...,[],[],[],[],[]
4,0,HB00002,,,,"AN ACT AMENDING REPUBLIC ACT NO. 9344, AS AMEN...",Reverts back the minimum age of criminal liabi...,Minimum Age Of Criminal Responsibility Act,2016-06-30,NATIONAL,Reform,JUSTICE,,"[CASTRO, FREDENIL ""FRED"" H., ALVAREZ, PANTALEO...","[Bataoil, Garcia (G.), Mercado, Cosalan, Anton...",[],[],[Gullas],[MOTHER BILL: HB08858],[REFERRAL TO THE COMMITTEE ON JUSTICE ON 2016-...,[],[],[],[],[]
5,0,HB00004,,,,AN ACT CREATING CIRCUIT CRIMINAL COURTS GRANTI...,Creates Circuit Criminal Courts in all judicia...,Circuit Criminal Courts Act Of 2016,2016-06-30,NATIONAL,Reform,JUSTICE,,"[FARIÑAS, RODOLFO C., ALVAREZ, PANTALEON D., A...","[Nieto, Bataoil, Villanueva, Sy-Alvarado, Espi...",[],[],[],[],[REFERRAL TO THE COMMITTEE ON JUSTICE ON 2016-...,[],[],[],[],[]
6,0,HB00001,,,,AN ACT IMPOSING THE DEATH PENALTY ON CERTAIN H...,Imposes death penalty for heinous crimes with ...,Death Penalty Law,2016-06-30,NATIONAL,Reform,JUSTICE,,"[CASTRO, FREDENIL ""FRED"" H., ALVAREZ, PANTALEO...","[Bataoil, Garcia (G.), Mercado, Cosalan, Yap (...",[],[],[Casilao],[MOTHER BILL: HB04727],[REFERRAL TO THE COMMITTEE ON JUSTICE ON 2016-...,[],[],[],[],[]
7,0,HB00006,,,,AN ACT ENHANCING PUBLIC SAFETY AT THE LOCAL LE...,Seeks to enhance public order and safety at th...,Public Safety Act,2016-06-30,NATIONAL,Reform,LOCAL GOVERNMENT,,"[FARIÑAS, RODOLFO C., ALVAREZ, PANTALEON D., A...","[Roque (H.), Nieto, Bataoil, Cosalan, Tan (A.)...",[],[],[],[MOTHER BILL: HB07878],[REFERRAL TO THE COMMITTEE ON LOCAL GOVERNMENT...,[],[],[],[],[]


In [8]:
#Save df to file
df.to_pickle("./df.pkl")

In [ ]:
#Read df from file
df = pd.read_pickle("./df.pkl")